# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
!pip install tabulate # https://stackoverflow.com/questions/18528533/pretty-printing-a-pandas-dataframe

In [2]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
from tabulate import tabulate

#### Creating list of filepaths to process original event csv data files

In [3]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/workspace/home


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [4]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [5]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [6]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [7]:
# COMPLETED: Create a Keyspace 
try:
    session.execute("""
        CREATE KEYSPACE IF NOT EXISTS udacity
        WITH REPLICATION = {
            'class': 'SimpleStrategy',
            'replication_factor':1
        }
    """)
except Exception as e:
    print(e)

print("KEYSPACE is created successfully!")

KEYSPACE is created successfully!


#### Set Keyspace

In [8]:
# COMPLETED: Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace("udacity")
    print("Session is now open with udacity keyspace, are now availble for query execution")
except Exception as e:
    print(e)
print("Done!")

Session is now open with udacity keyspace, are now availble for query execution
Done!


### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [9]:
## COMPLETED: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
query = """CREATE TABLE IF NOT EXISTS song_info_by_session ( 
        session_id int, 
        item_in_session int, 
        artist_name text, 
        song_title text, 
        song_length float, 
        PRIMARY KEY(session_id, item_in_session)
    )"""
try:
    rows = session.execute(query)
    print("Table has been created successfully!")
except Exception as e:
    print(e)
    
#print(type(rows._current_rows))
print(rows.current_rows)                    

Table has been created successfully!
[]


For table streaming_song_info `session_id` is the partition key and `item_in_session` is used as clustering column to help make up a unique key and the queries will filter by these columns

In [10]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'
count = 0
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## COMPLETED: Assign the INSERT statements into the `query` variable
        query = """INSERT 
                    INTO 
                    song_info_by_session (
                        session_id, 
                        item_in_session, 
                        artist_name, 
                        song_title, 
                        song_length
                    )"""
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        ## COMPLETED: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))
        count+=1

print(f"Totally,{count} records are inserted successfully!,")

Totally,6820 records are inserted successfully!,


#### Do a SELECT to verify that the data have been inserted into each table

In [11]:
## COMPLETED: Add in the SELECT statement to verify the data was entered into the table
query = """SELECT 
            artist_name, 
            song_title, 
            song_length 
        FROM song_info_by_session 
        WHERE session_id=338 
        AND item_in_session=4"""
try:
    rows = session.execute(query)
    print("Query executed successfully")
except Exception as e:
    print(e)
df = pd.DataFrame(rows._current_rows)
assert len(df) == 1 # Assertion is true, as there is only one record exist in the table, matching the given conditions
print(tabulate(df, headers='keys', tablefmt='psql'))

Query executed successfully
+----+---------------+---------------------------------+---------------+
|    | artist_name   | song_title                      |   song_length |
|----+---------------+---------------------------------+---------------|
|  0 | Faithless     | Music Matters (Mark Knight Dub) |       495.307 |
+----+---------------+---------------------------------+---------------+


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

#### Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [12]:
## COMPLETED: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
try:
    session.execute("""
        CREATE TABLE IF NOT EXISTS user_info_by_id_and_session (
            user_id INT,
            session_id INT,
            item_in_session INT,
            artist_name TEXT,
            song_title TEXT,
            first_name TEXT,
            last_name TEXT,
            PRIMARY KEY ((user_id, session_id), item_in_session)
        )
    """)
    print("Users table created successully!")
except Exception as error:
    print(error) 

          

Users table created successully!


For table `user`, the `user_id` and `session_id` are used as composite partition key because the queries will filter by these columns and `item_in_session` is used as clustering column to help make up a unique key.

In [13]:
file = 'event_datafile_new.csv'
count = 0
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = """INSERT 
                    INTO user_info_by_id_and_session (
                        user_id, 
                        session_id, 
                        item_in_session, 
                        artist_name, 
                        song_title, 
                        first_name, 
                        last_name
                    )"""
        query = query + """VALUES (%s, %s, %s, %s, %s, %s, %s)"""
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))
        count+=1
print(f"Totally {count} records inserted")

Totally 6820 records inserted


In [14]:
query = """
    SELECT 
        artist_name, 
        song_title, 
        first_name, 
        last_name 
    FROM user_info_by_id_and_session 
    WHERE user_id = 10 AND session_id = 182
"""
try:
    users_results = session.execute(query)
    print("Data retrieved successfully!")
except Exception as error:
    print(error)

df = pd.DataFrame(users_results._current_rows)
assert len(df) == 4 # user have watched 4 songs
print(tabulate(df, headers='keys', tablefmt='psql'))

Data retrieved successfully!
+----+-------------------+------------------------------------------------------+--------------+-------------+
|    | artist_name       | song_title                                           | first_name   | last_name   |
|----+-------------------+------------------------------------------------------+--------------+-------------|
|  0 | Down To The Bone  | Keep On Keepin' On                                   | Sylvie       | Cruz        |
|  1 | Three Drives      | Greece 2000                                          | Sylvie       | Cruz        |
|  2 | Sebastien Tellier | Kilometer                                            | Sylvie       | Cruz        |
|  3 | Lonnie Gordon     | Catch You Baby (Steve Pitron & Max Sanna Radio Edit) | Sylvie       | Cruz        |
+----+-------------------+------------------------------------------------------+--------------+-------------+


#### Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [15]:
## COMPLETED: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

try:
    session.execute("""
        CREATE TABLE IF NOT EXISTS user_info_by_song (
            song_title TEXT,
            user_id INT,
            first_name TEXT,
            last_name TEXT,
            PRIMARY KEY (song_title, user_id)
        )
    """)
except Exception as error:
    print(error)

print("Table has been created successfully")                    

Table has been created successfully


song_title and user_id has been used as composite partition key because the queries will filter by these columns and item_in _session is used as clustering column to help make a unique key

In [16]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = """INSERT 
                    INTO user_info_by_song (
                        song_title, 
                        user_id, 
                        first_name, 
                        last_name
                    ) VALUES (%s, %s, %s, %s)"""
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))
print("Data ingestion from CSV  file to database is completed successully!")                    

Data ingestion from CSV  file to database is completed successully!


In [17]:
query = """
    SELECT first_name, last_name FROM user_info_by_song 
    WHERE song_title = 'All Hands Against His Own'
"""
try:
    results = session.execute(query)
except Exception as error:
    print(error)
df = pd.DataFrame(results._current_rows)
print(tabulate(df, headers='keys', tablefmt='psql')) # lists the first 5 records of the dataframe (df).

+----+--------------+-------------+
|    | first_name   | last_name   |
|----+--------------+-------------|
|  0 | Jacqueline   | Lynch       |
|  1 | Tegan        | Levine      |
|  2 | Sara         | Johnson     |
+----+--------------+-------------+


### Drop the tables before closing out the sessions

In [18]:
## COMPLETED: Drop the table before closing out the sessions
try:
    session.execute("DROP TABLE IF EXISTS song_info_by_session")
    session.execute("DROP TABLE IF EXISTS user_info_by_id_and_session")
    session.execute("DROP TABLE IF EXISTS user_info_by_song")
except Exception as error:
    print(error)  
    
print("Tables are successfully droped!")

Tables are successfully droped!


### Close the session and cluster connection¶

In [19]:
session.shutdown()
print("@Session is closed successfully!")
cluster.shutdown()
print("@Cluster is shutdown successfully!")

@Session is closed successfully!
@Cluster is shutdown successfully!
